In [1]:
from selenium import webdriver
import json
import time

# Load saved cookies from the JSON file
with open('cookies.json', 'r') as file:
    cookies = json.load(file)

# Start a new Firefox WebDriver session
driver = webdriver.Firefox()

# Open the base domain (needed to add cookies)
driver.get('https://pony.town')

# Add the cookies to the current session
for cookie in cookies:
    if 'sameSite' in cookie and cookie['sameSite'] not in ["Strict", "Lax", "None"]:
        cookie['sameSite'] = 'None'  # Adjust 'sameSite' attribute if necessary
    try:
        driver.add_cookie(cookie)
    except Exception as e:
        print(f"Error adding cookie: {cookie['name']}, {e}")

# Navigate to the specific page (search query)
driver.get('https://pony.town/')

# Allow time for the page to load and the session to activate
time.sleep(20)

# Scroll down the page
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

# Allow time for the page to load additional content
time.sleep(5)

# Save the HTML content to a file
html_content = driver.page_source
with open('page_content.html', 'w', encoding='utf-8') as file:
    file.write(html_content)

# Take a screenshot as proof of session reuse
driver.save_screenshot('session_with_cookies.png')

# Quit the browser
driver.quit()